# Below 8 line of code are for to download dataset directly to the google drive and to unzip it. This step saves a lot of time for those using google collab on a regular basis.

In [ ]:
# import json
# import os

In [ ]:
# file = open('/content/drive/MyDrive/datasets/cotton_disease/Cotton Disease/train/kaggle.json')
# f = json.load(file)

In [ ]:
# f

In [ ]:
# os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/My Drive/kaggle'

In [ ]:
# %cd /content/drive/My Drive/kaggle

In [ ]:
# os.environ['KAGGLE_USERNAME'] = "abhishekkumarsaini" # username from the json file
# os.environ['KAGGLE_KEY'] = "ed58dc4d33dd5ed05e33cf59aa9266ee" # key from the json file

In [ ]:
# !kaggle datasets download -d fedesoriano/stroke-prediction-dataset

In [ ]:
# !unzip \*.zip && rm *.zip 

# Importing some important libraries, loading the dataset and gaining some basic insights about the data. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
data.head()

In [ ]:
data = data.drop('id', axis=1)

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
columns = ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke']

Seperating the categorical features from the other features and finding out the count of categories.

In [ ]:
for c in columns:
  if data[c].dtype == 'object':
    print(data[c].value_counts())
    print(" ")

In [ ]:
data[data.bmi == 97.600]

In [ ]:
data = data.dropna(axis=0)

In [ ]:
data.info()

 Droping only one value that is gender[other] which have only one value.

In [ ]:
data.drop(data[data.gender == 'Other'].index, inplace = True)

# **Exploratory data analysis uisng seaborn library.**

This shows that our dataset is very unbalanced. We should keep this in mind while building our model and selecting the performance matrics. We can do undersampling or oversampling for the unbalanced dataset.

In [ ]:
data['stroke'].value_counts().plot(kind = 'pie', autopct="%.2f", figsize=(6,6))

In [ ]:
sns.catplot(x='stroke',y='age',data= data)

This shows that most of the people getting the stroke are from the age group of 60 to 80.

In [ ]:
sns.catplot(x='stroke', y='age', kind= 'box', data=data)

In [ ]:
sns.catplot(y="age", x="stroke", hue="gender",
            kind="violin", data=data)

In [ ]:
sns.catplot(x="Residence_type", kind="count", palette="ch:.25", data=data, hue='stroke')

Conclusions:
1. people living in urban are getting more stroke.
2. people in private sector getting more strokes.


In [ ]:
for c in columns:
  if data[c].dtype == 'object':
    sns.catplot(x='stroke', kind='count',hue=c,data=data)

In [ ]:
corr = data.corr()
corr['stroke'].sort_values(ascending= False)

# Data preprocessing.

Listing out all categorical columns.

In [ ]:
categorical_columns = []
for c in columns:
  if data[c].dtype == 'object':
    categorical_columns.append(c)


In [ ]:
categorical_columns

Train_Test split using stratified sampling. I used stratified sampling so that both train and test data gets same ratio of positive and negative class data.

---



In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train , X_test, Y_train, Y_test = train_test_split(data, data['stroke'], test_size=.25, stratify=data['stroke'])

In [ ]:
X_train['stroke'].value_counts()

In [ ]:
X_test['stroke'].value_counts()

In [ ]:
X_train = X_train.drop('stroke', axis=1)

In [ ]:
X_test = X_test.drop('stroke', axis=1)

In [ ]:
numerical_columns = []
for c in X_train.columns:
  if X_train[c].dtype in ['float64','int64']:
    numerical_columns.append(c)

In [ ]:
numerical_columns

Creating a function that will convert all categorical features into the dummy variables. It is like one_hot encoding.

In [ ]:
def make_dummy(b, colm):
  return pd.get_dummies(b, columns= colm)

In [ ]:
X_train = make_dummy(X_train, categorical_columns)

In [ ]:
X_train.shape

In [ ]:
X_test = make_dummy(X_test, categorical_columns)

In [ ]:
X_test.shape

Standardizing both test and train data so that model training becomes fast.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaller = StandardScaler() 

In [ ]:
X_train = scaller.fit_transform(X_train)
X_test = scaller.transform(X_test)

In [ ]:
Y_true_train = Y_train==1
Y_true_test = Y_test==1

Below what we are going to do is that training different classifier model and will compute recall value. I have selected the recall as the performance matrics because no person should me miss diagnosed as he/she will not  have stroke.
# note 
first i will do this step before doing any type of sampling. 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [ ]:
models = {'dt_clf':DecisionTreeClassifier(), 'for_clf':RandomForestClassifier(), 'k_clf':KNeighborsClassifier(), 'logistic_reg':LogisticRegression(), 'ada_boost': AdaBoostClassifier()}

In [ ]:
results = []
for key in models:
  model_name = models[key]
  model = model_name
  model.fit(X_train, Y_train)
  pred = model.predict(X_test)

  # pred = model_name.predict(X_test)
  print('')
  print(key)
  print(confusion_matrix(Y_true_test, pred),'recall=', metrics.recall_score(Y_true_test, pred))

Performing oversampling. I have selected oversampling because the amount of data available is very less.

In [ ]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss

In [ ]:
smk = SMOTETomek(random_state=42)
X_res,Y_res=smk.fit_resample(X_train,Y_train)

In [ ]:
Y_true_train_res = Y_res ==1

In [ ]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(Y_train)))
print('Resampled dataset shape {}'.format(Counter(Y_res)))

This shows that logistic regression and ada boost is having descent recall score.

In [ ]:
results = []
for key in models:
  model_name = models[key]
  model = model_name
  model.fit(X_res, Y_res)
  pred = model.predict(X_test)

  # pred = model_name.predict(X_test)
  print('')
  print(key)
  print(confusion_matrix(Y_true_test, pred),'recall=', metrics.recall_score(Y_true_test, pred))

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import cross_val_predict

In [ ]:
ada = AdaBoostClassifier()
log_reg = LogisticRegression()

In [ ]:
ada.fit(X_res, Y_res)

In [ ]:
log_reg.fit(X_res, Y_res)

decision function of the model returns the probabilities.

In [ ]:
y_score = cross_val_predict(log_reg, X_res, Y_true_train_res, cv=3, method='decision_function')

In [ ]:
y_score.shape

Plotting precision and recall against threshold values for logistic regression model. 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
precisions, recalls, thresholds = precision_recall_curve(Y_true_train_res, y_score)

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
  plt.plot(thresholds, precisions[:-1], 'b--', label = 'precision')
  plt.plot(thresholds, recalls[:-1], 'g-', label = 'recall')
  [...]
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()  


Plotting precision and recall against threshold values for logistic regression model. 

In [ ]:
y_score_ada = cross_val_predict(ada, X_res, Y_true_train_res, cv=3, method='decision_function')
precisions_ada, recalls_ada, thresholds_ada = precision_recall_curve(Y_true_train_res, y_score_ada)
def plot_precision_recall_vs_threshold(precisions_ada, recalls_ada, thresholds_ada):
  plt.plot(thresholds_ada, precisions_ada[:-1], 'b--', label = 'precision')
  plt.plot(thresholds_ada, recalls_ada[:-1], 'g-', label = 'recall')
  [...]
plot_precision_recall_vs_threshold(precisions_ada, recalls_ada, thresholds_ada)
plt.show()  

In [ ]:
threshold = 0
log_scores = log_reg.decision_function(X_res)
log_pred_threshold = log_scores>threshold

In [ ]:
threshold_2 = 0
ada_scores = ada.decision_function(X_res)
ada_pred_threshold = ada_scores>threshold_2

In [ ]:
ada_pred_threshold

In [ ]:
log_pred_threshold

calculating precision and recall score on logistic regression model with threshold being 0 that is selected from the precision - recall curve against the different threshold values. 

In [ ]:
metrics.recall_score(Y_true_train_res, log_pred_threshold)

In [ ]:
metrics.precision_score(Y_true_train_res, log_pred_threshold)

In [ ]:
metrics.confusion_matrix(Y_true_train_res, log_pred_threshold)

calculating precision and recall score on ada boost model model with threshold being 0 that is selected from the precision - recall curve against the different threshold values.

In [ ]:
metrics.recall_score(Y_true_train_res, ada_pred_threshold)

In [ ]:
metrics.precision_score(Y_true_train_res, ada_pred_threshold)

In [ ]:
metrics.confusion_matrix(Y_true_train_res, ada_pred_threshold)

From above testing, scores of ada boost classifier model is very promising. It is giving descent recall and precision score.

Now let's evaluate our models on test data too.

In [ ]:
threshold_2 = 0
ada_scores_test = ada.decision_function(X_test)
ada_pred_test_threshold = ada_scores_test>threshold

In [ ]:
metrics.confusion_matrix(Y_true_test, ada_pred_test_threshold)

In [ ]:
metrics.recall_score(Y_true_test, ada_pred_test_threshold)

In [ ]:
metrics.precision_score(Y_true_test, ada_pred_test_threshold)

In [ ]:
threshold_2 = 0
log_scores_test = log_reg.decision_function(X_test)
log_pred_test_threshold = log_scores_test>threshold_2

In [ ]:
metrics.recall_score(Y_true_test, log_pred_test_threshold)

In [ ]:
metrics.precision_score(Y_true_test, log_pred_test_threshold)

Conclusions:
It seems that our ada boost clasifier was over fitting over our training data, whereas logistics regression model perfomed well comparatively.
We can tune our model to achieve better recall and precision score.

# Any suggestions and corrections are warmly welcomed.
# Thankyou!